In [ ]:
74%

from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    """Expanded CNN with more convolutional layers for complex image features learning."""
    def __init__(self, num_classes=2):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            # Layer 1
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Layer 2
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Layer 3
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Layer 4
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Adjusting the fully connected layers to match the output of the final pooling layer
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256),  # Adjust the input features according to the output from the last MaxPool
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# Cài đặt transform
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.Grayscale(num_output_channels=1),  # Chỉ sử dụng nếu ảnh là RGB
    transforms.ToTensor()
])

# # Tải dataset
# dataset_train = datasets.ImageFolder(root='data/train', transform=transform)
# train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)

# dataset_test = datasets.ImageFolder(root='data/test', transform=transform)
# test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

# dataset_val = datasets.ImageFolder(root='data/val', transform=transform)
# val_loader = DataLoader(dataset_val, batch_size=32, shuffle=False)
from torch.utils.data import random_split

# Load the dataset
dataset = datasets.ImageFolder(root='data/train', transform=transform)

# Calculate the sizes for training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=12)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=12)

# Load the test dataset
dataset_test = datasets.ImageFolder(root='data/test', transform=transform)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=12)


from tqdm import tqdm

def train_model(model, data_loader, epochs=30):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(data_loader, desc=f"Epoch {epoch+1}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(data_loader)}')
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = correct / total
        print(f'Epoch {epoch+1}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy * 100:.2f}%')


# Tạo và huấn luyện mô hình
model = Net()
train_model(model, train_loader)



#Test print classfications resport
def test_model(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    print(f'Accuracy: {correct/total}')
    return y_true, y_pred

y_true, y_pred = test_model(model, test_loader)

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))


## Dùng BCEWithLogits Loss

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from sklearn.metrics import classification_report

# Định nghĩa lại mô hình với một đầu ra duy nhất cho nn.BCEWithLogitsLoss
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Đổi đầu ra thành 1 đơn vị cho BCEWithLogitsLoss
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256),
            nn.ReLU(),
            nn.Linear(256, 1)  # Chuyển đổi thành 1 đầu ra
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [8]:
from torch.utils.tensorboard import SummaryWriter


In [9]:
writer = SummaryWriter("runs/my_classification_project")


In [5]:
# Cài đặt transform
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

# Load và chia dữ liệu
dataset = datasets.ImageFolder(root='data/train', transform=transform)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=12)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=12)

dataset_test = datasets.ImageFolder(root='data/test', transform=transform)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=12)

In [13]:
# Huấn luyện với nn.BCEWithLogitsLoss
def train_model(model, train_loader, val_loader, epochs=30):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)  # Chuyển labels thành float

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                # Chuyển logits sang xác suất bằng sigmoid và tính độ chính xác
                predicted = (torch.sigmoid(outputs) > 0.5).float()  # Dự đoán dựa trên ngưỡng 0.5
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = correct / total
        writer.add_scalar("Loss/Validation", val_loss, epoch)
        writer.add_scalar("Accuracy/Validation", val_accuracy, epoch)
        print(f'Epoch {epoch+1}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy * 100:.2f}%')

# Tạo và huấn luyện mô hình
model = Net()
train_model(model, train_loader, val_loader)


Epoch 1:   0%|          | 0/147 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 147/147 [00:23<00:00,  6.16it/s]

Epoch 1, Loss: 0.20814310645266454


Epoch 1, Validation Loss: 0.07824945712790769, Validation Accuracy: 96.93%


Epoch 2: 100%|██████████| 147/147 [00:24<00:00,  6.06it/s]

...

Epoch 29, Loss: 1.5640642268570965e-05


Epoch 29, Validation Loss: 0.07458499054974803, Validation Accuracy: 98.47%


Epoch 30: 100%|██████████| 147/147 [00:25<00:00,  5.73it/s]

Epoch 30, Loss: 1.6196282367738856e-05


Epoch 30, Validation Loss: 0.0719805943503084, Validation Accuracy: 98.47%


In [14]:
!tensorboard --logdir=runs


TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [15]:
# Đánh giá mô hình trên tập test
def test_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            predicted = (torch.sigmoid(outputs) > 0.5).float()  # Áp dụng sigmoid và ngưỡng 0.5 để phân loại
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    print(f'Accuracy: {correct / total * 100:.2f}%')
    return y_true, y_pred

# Chạy kiểm tra trên tập test
y_true, y_pred = test_model(model, test_loader)
print(classification_report(y_true, y_pred))

Accuracy: 75.00%
              precision    recall  f1-score   support

         0.0       0.99      0.34      0.50       234
         1.0       0.72      1.00      0.83       390

    accuracy                           0.75       624
   macro avg       0.85      0.67      0.67       624
weighted avg       0.82      0.75      0.71       624

